In [ ]:
import os
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
home_dir= "/home2/s439906/project/CASP16/oligomer/"
os.chdir(home_dir)
raw_data_dir = "./oligomer_data_Nov_17/raw_data/"
out_dir = "./check_oligomer/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

files = [file for file in os.listdir(raw_data_dir) if file.endswith(".csv")]
files.sort()
files

In [ ]:
scores=[
                        # 'QSbest', 
                        'ICS(F1)',
                        'lDDT',
                        # 'DockQ_Avg',
                        'IPS(JaccCoef)',
                          'TMscore', 
                    ]
# weights=[
#                        1/4, 1/4, 1/4,
#                           1/4,
#                             1/6, 1/6,
#                     ]
# weights = [1/4, 1/4, 1/4, 1/4]
weights = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
score_df = None
for i in range(len(scores)):
    score = scores[i]
    weight = weights[i]
    large_data = None
    for file in files:
        target = file.split(".")[0]
        df = pd.read_csv(raw_data_dir+file, index_col=0)
        df_score = df[score]
        score_dict = {}
        for key, value in df_score.items():
            key, model_id = key.split("_")
            model_id = str(model_id)[0]
            _, group = key.split("TS")
            group = "TS" + group
            # print(target, score, key, value)
            try:
                value = float(value)
                # if it is nan, print it
                if value != value:
                    print(target, score, key, value)
                    # if score == "MolPrb_Score":
                    #     value = -99.99
                    # else:
                    #     value = 0.0
            except:
                print(target, score, key, value)
                # value = 0.0     
            if model_id != '6':
                if group not in score_dict:
                    score_dict[group] = value
                else:
                    if value > score_dict[group]:
                        score_dict[group] = value  
        print(target, score, score_dict)
        data = pd.DataFrame.from_dict(score_dict, orient='index')
        # change the column name to target name
        data.columns = [target]
        mean_initial = data[target].mean()
        std_initial = data[target].std()
        data['z_score_initial'] = (data[target] - mean_initial) / std_initial
        filtered_data = data[data['z_score_initial'] >= -2]
        mean_recomputed = filtered_data[target].mean()
        std_recomputed = filtered_data[target].std()
        data['z_score_recomputed'] = (data[target] - mean_recomputed) / std_recomputed
        data.loc[data['z_score_recomputed'] < 0, 'z_score_recomputed'] = 0
        print(data)
        data_final = data[['z_score_recomputed']]
        data_final.columns = [target]
        if large_data is None:
            large_data = data_final
        else:
            large_data = pd.concat([large_data, data_final], axis=1)
    # impute the missing values again with -2
    large_data = large_data.fillna(0)
    # sort the columns and rows
    large_data = large_data.sort_index()
    large_data = large_data.sort_index(axis=1)
    large_data.to_csv(out_dir+score+".csv")
    large_data = large_data * weight
    if score_df is None:
        score_df = large_data
    else:
        # add the data to the score_df
        score_df += large_data



In [ ]:
score_df.to_csv(out_dir+"final_score.csv")
# data_columns = score_df.columns
# target_count = {}
# for EU in data_columns:
#     target = EU.split("-")[0]
#     if target not in target_count:
#         target_count[target] = 0
#     target_count[target] += 1
# target_weight = {key: 1/value for key, value in target_count.items()}
# EU_weight = {EU: target_weight[EU.split("-")[0]]
#              for EU in data_columns}
# EU_weight.__len__()
# EU_weight = pd.Series(EU_weight)
# score_df = score_df * EU_weight
# sort score_df by the sum of the row
score_df['sum'] = score_df.sum(axis=1)
score_df = score_df.sort_values('sum', ascending=False)
score_df_sum = score_df['sum']
print(score_df_sum.head(20))
print(score_df_sum)
# save score_df_sum
score_df_sum.to_csv(out_dir+"final_score_sum.csv")

In [ ]:
scores=[
                        'QSbest', 
                        'ICS(F1)',
                        # 'lDDT',
                        'DockQ_Avg',
                        'IPS(JaccCoef)',
                        #   'TMscore', 
                    ]
weights=[
                       1/6, 1/6, 1/6,
                          1/6,
                            1/6, 1/6,
                    ]
score_df = None
for i in range(len(scores)):
    score = scores[i]
    weight = weights[i]
    large_data = None
    for file in files:
        target = file.split(".")[0]
        df = pd.read_csv(raw_data_dir+file, index_col=0)
        df_score = df[[score]]
        # convert to float
        df_score = df_score.astype(float)
        initial_mean = df_score.mean()
        initial_std = df_score.std()
        initial_z_score = (df_score - initial_mean) / initial_std
        # compute new mean and std using z_score > -2
        filtered_data = df_score[initial_z_score >= -2]
        recomputed_mean = filtered_data.mean()
        recomputed_std = filtered_data.std()
        recomputed_z_score = (df_score - recomputed_mean) / recomputed_std
        recomputed_z_score[recomputed_z_score < 0] = 0
        recomputed_z_score = recomputed_z_score [score]
        score_dict ={}
        for key, value in recomputed_z_score.items():
            key, model_id = key.split("_")
            model_id = str(model_id)[0]
            _, group = key.split("TS")
            group = "TS" + group
            try:
                value = float(value)
                if value != value:
                    print(target, score, key, value)
            except:
                continue
            if model_id != '6':
                if group not in score_dict:
                    score_dict[group] = value
                else:
                    if value > score_dict[group]:
                        score_dict[group] = value        
        data = pd.DataFrame.from_dict(score_dict, orient='index')
        data.columns = [target]
        if large_data is None:
            large_data = data
        else:
            large_data = pd.concat([large_data, data], axis=1)
    # impute the missing values again with -2
    large_data = large_data.fillna(0)
    # sort the columns and rows
    large_data = large_data.sort_index()
    large_data = large_data.sort_index(axis=1)
    large_data.to_csv(out_dir+score+".csv")
    large_data = large_data * weight
    if score_df is None:
        score_df = large_data
    else:
        # add the data to the score_df
        score_df += large_data
score_df
score_df.to_csv(out_dir+"final_score.csv")
# data_columns = score_df.columns
# target_count = {}
# for EU in data_columns:
#     target = EU.split("-")[0]
#     if target not in target_count:
#         target_count[target] = 0
#     target_count[target] += 1
# target_weight = {key: 1/value for key, value in target_count.items()}
# EU_weight = {EU: target_weight[EU.split("-")[0]]
#              for EU in data_columns}
# EU_weight.__len__()
# EU_weight = pd.Series(EU_weight)
# score_df = score_df * EU_weight
# sort score_df by the sum of the row
score_df['sum'] = score_df.sum(axis=1)
score_df = score_df.sort_values('sum', ascending=False)
score_df_sum = score_df['sum']
print(score_df_sum.head(20))
print(score_df_sum)

In [ ]:

score_df = None
for i in range(len(scores)):
    score = scores[i]
    weight = weights[i]
    large_data = None
    for file in files:
        target = file.split(".")[0]
        df = pd.read_csv(raw_data_dir+file, index_col=0)
        df_score = df[score]
        if score == "MolPrb_Score":
            # invert the score
            df_score = - df_score
        # convert to dict
        df_score = df_score.to_dict()
        score_dict = {}
        for key, value in df_score.items():
            key, domain = key.split("-")
            key, model_id = key.split("_")
            _, group = key.split("TS")
            group = "TS" + group
            try:
                value = float(value)
                # if it is nan, print it
                if value != value:
                    # print(target, score, key, value)
                    if score == "MolPrb_Score":
                        value = -99.99
                    else:
                        value = 0.0
            except:
                print(target, score, key, value)
                value = 0.0
            if model_id != '6':
                if group not in score_dict:
                    score_dict[group] = value
                else:
                    if value > score_dict[group]:
                        score_dict[group] = value
        print(target, score, score_dict)
        data = pd.DataFrame.from_dict(score_dict, orient='index')
        # change the column name to target name
        data.columns = [target]
        mean_initial = data[target].mean()
        std_initial = data[target].std()
        data['z_score_initial'] = (data[target] - mean_initial) / std_initial
        filtered_data = data[data['z_score_initial'] >= -2]
        mean_recomputed = filtered_data[target].mean()
        std_recomputed = filtered_data[target].std()
        data['z_score_recomputed'] = (data[target] - mean_recomputed) / std_recomputed
        data.loc[data['z_score_recomputed'] < -2, 'z_score_recomputed'] = -2
        print(data)
        data_final = data[['z_score_recomputed']]
        data_final.columns = [target]
        if large_data is None:
            large_data = data_final
        else:
            large_data = pd.concat([large_data, data_final], axis=1)
    
    # impute the missing values again with -2
    large_data = large_data.fillna(-2)
    # sort the columns and rows
    large_data = large_data.sort_index()
    large_data = large_data.sort_index(axis=1)
    large_data.to_csv(out_dir+score+".csv")
    large_data = large_data * weight
    if score_df is None:
        score_df = large_data
    else:
        # add the data to the score_df
        score_df += large_data

score_df.to_csv(out_dir+"final_score.csv")
data_columns = score_df.columns
target_count = {}
for EU in data_columns:
    target = EU.split("-")[0]
    if target not in target_count:
        target_count[target] = 0
    target_count[target] += 1
target_weight = {key: 1/value for key, value in target_count.items()}
EU_weight = {EU: target_weight[EU.split("-")[0]]
             for EU in data_columns}
EU_weight.__len__()
EU_weight = pd.Series(EU_weight)
score_df = score_df * EU_weight
# sort score_df by the sum of the row
score_df['sum'] = score_df.sum(axis=1)
score_df = score_df.sort_values('sum', ascending=False)
score_df_sum = score_df['sum']
print(score_df_sum.head(20))

In [ ]:
home_dir= "/home2/s439906/project/CASP16/monomer/"
os.chdir(home_dir)
raw_data_dir = "./monomer_data_Sep_15_EU/raw_data/"
out_dir = "./check_with_qian/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

files = [file for file in os.listdir(raw_data_dir) if file.startswith("T1") and file.endswith(".csv")]
# sort files alphabetically
files.sort()
scores=[
                        'AL0_P', 'SphGr',
                        'MolPrb_Score',
                        'GDT_HA', 'GDC_SC', 'reLLG_const',
                        'CAD_AA', 'LDDT',
                        'QSE',
                    ]
weights=[
                        1/16, 1/8,
                        1/16,
                        1/6, 1/16, 1/6,
                        1/8, 1/16,
                        1/6,
                    ]

score_df = None
for i in range(len(scores)):
    score = scores[i]
    weight = weights[i]
    large_data = None
    for file in files:
        target = file.split(".")[0]
        df = pd.read_csv(raw_data_dir+file, index_col=0)
        df_score = df[score]
        if score == "MolPrb_Score":
            # invert the score
            df_score = - df_score
        # convert to dict
        df_score = df_score.to_dict()
        score_dict = {}
        for key, value in df_score.items():
            key, domain = key.split("-")
            key, model_id = key.split("_")
            _, group = key.split("TS")
            group = "TS" + group
            try:
                value = float(value)
                # if it is nan, print it
                if value != value:
                    # print(target, score, key, value)
                    if score == "MolPrb_Score":
                        value = -99.99
                    else:
                        value = 0.0
            except:
                print(target, score, key, value)
                value = 0.0
            if model_id == '1':
                if group not in score_dict:
                    score_dict[group] = value
                else:
                    if value > score_dict[group]:
                        score_dict[group] = value
        print(target, score, score_dict)
        data = pd.DataFrame.from_dict(score_dict, orient='index')
        # change the column name to target name
        data.columns = [target]
        mean_initial = data[target].mean()
        std_initial = data[target].std()
        data['z_score_initial'] = (data[target] - mean_initial) / std_initial
        filtered_data = data[data['z_score_initial'] >= -2]
        mean_recomputed = filtered_data[target].mean()
        std_recomputed = filtered_data[target].std()
        data['z_score_recomputed'] = (data[target] - mean_recomputed) / std_recomputed
        data.loc[data['z_score_recomputed'] < -2, 'z_score_recomputed'] = -2
        print(data)
        data_final = data[['z_score_recomputed']]
        data_final.columns = [target]
        if large_data is None:
            large_data = data_final
        else:
            large_data = pd.concat([large_data, data_final], axis=1)
    
    # impute the missing values again with -2
    large_data = large_data.fillna(-2)
    # sort the columns and rows
    large_data = large_data.sort_index()
    large_data = large_data.sort_index(axis=1)
    large_data.to_csv(out_dir+score+".csv")
    large_data = large_data * weight
    if score_df is None:
        score_df = large_data
    else:
        # add the data to the score_df
        score_df += large_data

score_df.to_csv(out_dir+"final_score.csv")

data_columns = score_df.columns
target_count = {}
for EU in data_columns:
    target = EU.split("-")[0]
    if target not in target_count:
        target_count[target] = 0
    target_count[target] += 1
target_weight = {key: 1/value for key, value in target_count.items()}
EU_weight = {EU: target_weight[EU.split("-")[0]]
             for EU in data_columns}
EU_weight.__len__()
EU_weight = pd.Series(EU_weight)
score_df = score_df * EU_weight
# sort score_df by the sum of the row
score_df['sum'] = score_df.sum(axis=1)
score_df = score_df.sort_values('sum', ascending=False)
score_df_sum = score_df['sum']
print(score_df_sum.head(20))